In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras


In [ ]:
import pandas as pd

# Define the file path
file_path = '/content/Devil_Wears_Prada.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())


  Speaker                                           Dialogue
0    andy       What floor is Elias-Clarke? Human Resources.
1   guard                         Honey, you want West 57th.
2    andy  This article was about the Take Back the Night...
3  sherry  I have two positions available. One is assista...
4    andy                               And the other one--?


In [ ]:
df.head()

,Speaker,Dialogue
0,andy,What floor is Elias-Clarke? Human Resources.
1,guard,"Honey, you want West 57th."
2,andy,This article was about the Take Back the Night...
3,sherry,I have two positions available. One is assista...
4,andy,And the other one--?


In [ ]:
df.shape

(673, 2)

In [ ]:
df.drop(columns = 'Speaker',inplace=True)

In [ ]:
train_text = '\n'.join(df.iloc[:538, 0].astype(str))  # Joining first 538 rows for training
test_text = '\n'.join(df.iloc[-135:, 0].astype(str))  # Joining last 135 rows for testing

In [ ]:
train_text

'What floor is Elias-Clarke? Human Resources.\nHoney, you want West 57th.\nThis article was about the Take Back the Night march. This was a four-part series on the impact of busing on the public schools...\nI have two positions available. One is assistant to the road test editor of Auto Universe magazine.\n And the other one--?\nHi, I have an appointment with Emily Charlton--\nOkay, so... I was Miranda\'s second assistant, but her first assistant recently got promoted so now I\'m the first...\nAnd you\'re replacing yourself.\nI\'m trying. Miranda sacked the last two girls after only a few weeks. We need to find someone who can survive here. Do you understand?\nYes. Of course. Who\'s Miranda?\n You didn\'t just ask me that. She\'s the editor in chief of Runway. Not to mention a legend. Work a year for her and you can get a job at any magazine you want. A million girls would kill for this job.\nSounds great. I\'d love to be considered.\nAndrea, Runway is a fashion magazine. An interest i

In [ ]:
test_text

'You don\'t mean that.\nI do. I really do.\nThen maybe this trip is coming at a good time. Maybe we should take a break.\nIn case you\'re wondering, the person whose calls you always take, that\'s the relationship you\'re in.\nHello, Miranda...\nWait. This is not right. Miranda is staying in the suite.\nThat\'s correct, Mademoiselle. Miranda Priestly\'s suite is down the hall.\nThis is my room?\nYou know, I\'ve been thinking, and you still owe me for Harry Potter.\nOh, do I?\nOf course you do. You working tonight?\nActually, Miranda has a dinner.\nSo you\'re free. Perfect. Oh, but there\'s a problem, huh? Le Boyfriend.\nOh, you\'re so full of it. You are not desol at all.\nYeah, not even a little. What time should I pick you up?\n There you are. We need to go over the seating chart for the luncheon.\nSure. No problem. I have it right here, um...\nBy all means, move at a glacial pace. You know how that thrills me.\nBut your table is full.\nStephen won\'t be coming.\nSo Stephen is not...

In [ ]:
train_text = train_text.lower()
test_text = test_text.lower()

In [ ]:
train_text

'what floor is elias-clarke? human resources.\nhoney, you want west 57th.\nthis article was about the take back the night march. this was a four-part series on the impact of busing on the public schools...\ni have two positions available. one is assistant to the road test editor of auto universe magazine.\n and the other one--?\nhi, i have an appointment with emily charlton--\nokay, so... i was miranda\'s second assistant, but her first assistant recently got promoted so now i\'m the first...\nand you\'re replacing yourself.\ni\'m trying. miranda sacked the last two girls after only a few weeks. we need to find someone who can survive here. do you understand?\nyes. of course. who\'s miranda?\n you didn\'t just ask me that. she\'s the editor in chief of runway. not to mention a legend. work a year for her and you can get a job at any magazine you want. a million girls would kill for this job.\nsounds great. i\'d love to be considered.\nandrea, runway is a fashion magazine. an interest i

In [ ]:
test_text

'you don\'t mean that.\ni do. i really do.\nthen maybe this trip is coming at a good time. maybe we should take a break.\nin case you\'re wondering, the person whose calls you always take, that\'s the relationship you\'re in.\nhello, miranda...\nwait. this is not right. miranda is staying in the suite.\nthat\'s correct, mademoiselle. miranda priestly\'s suite is down the hall.\nthis is my room?\nyou know, i\'ve been thinking, and you still owe me for harry potter.\noh, do i?\nof course you do. you working tonight?\nactually, miranda has a dinner.\nso you\'re free. perfect. oh, but there\'s a problem, huh? le boyfriend.\noh, you\'re so full of it. you are not desol at all.\nyeah, not even a little. what time should i pick you up?\n there you are. we need to go over the seating chart for the luncheon.\nsure. no problem. i have it right here, um...\nby all means, move at a glacial pace. you know how that thrills me.\nbut your table is full.\nstephen won\'t be coming.\nso stephen is not...

In [ ]:
import re

train_text = re.sub(r'[^a-zA-Z\s]', '', train_text)

In [ ]:
test_text = re.sub(r'[^a-zA-Z\s]', '', test_text)

In [ ]:
train_text = re.sub(r'<.*?>', '', train_text)  # Remove HTML tags
train_text = re.sub(r'\[[^\]]*\]', '', train_text)  # Remove text within square brackets

In [ ]:
test_text = re.sub(r'<.*?>', '', test_text)  # Remove HTML tags
test_text = re.sub(r'\[[^\]]*\]', '', test_text)  # Remove text within square brackets

In [ ]:
len(train_text)

29488

In [ ]:
len(test_text)

7166

In [ ]:
import nltk

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
tokenizer = Tokenizer()
#fit
tokenizer.fit_on_texts([train_text])
#assign length of word index
total_words_train = len(tokenizer.word_index) + 1

In [ ]:
total_words_train

1318

In [ ]:
tokenizer.fit_on_texts([test_text])
#assign length of word index
total_words_test = len(tokenizer.word_index) + 1

In [ ]:
total_words_test

1475

In [ ]:
tokenizer.word_index

{'i': 1,
 'you': 2,
 'the': 3,
 'to': 4,
 'a': 5,
 'and': 6,
 'of': 7,
 'is': 8,
 'that': 9,
 'me': 10,
 'for': 11,
 'in': 12,
 'it': 13,
 'what': 14,
 'have': 15,
 'im': 16,
 'no': 17,
 'miranda': 18,
 'do': 19,
 'not': 20,
 'dont': 21,
 'my': 22,
 'this': 23,
 'her': 24,
 'youre': 25,
 'its': 26,
 'on': 27,
 'so': 28,
 'but': 29,
 'be': 30,
 'she': 31,
 'about': 32,
 'are': 33,
 'your': 34,
 'know': 35,
 'with': 36,
 'at': 37,
 'thats': 38,
 'was': 39,
 'we': 40,
 'just': 41,
 'get': 42,
 'all': 43,
 'can': 44,
 'like': 45,
 'go': 46,
 'going': 47,
 'need': 48,
 'job': 49,
 'oh': 50,
 'want': 51,
 'if': 52,
 'see': 53,
 'right': 54,
 'from': 55,
 'there': 56,
 'one': 57,
 'think': 58,
 'did': 59,
 'up': 60,
 'look': 61,
 'out': 62,
 'okay': 63,
 'would': 64,
 'now': 65,
 'here': 66,
 'were': 67,
 'andy': 68,
 'new': 69,
 'ill': 70,
 'an': 71,
 'emily': 72,
 'girls': 73,
 'runway': 74,
 'sure': 75,
 'work': 76,
 'great': 77,
 'god': 78,
 'sorry': 79,
 'why': 80,
 'course': 81,
 'shes'

In [ ]:
input_sequences_train = []
#split the sentence from '\n'
for line in train_text.split('\n'):
    #get tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences_train.append(n_gram_sequence)

In [ ]:
input_sequences_test = []
#split the sentence from '\n'
for line in test_text.split('\n'):
    #get tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences_test.append(n_gram_sequence)

In [ ]:
input_sequences_train

[[14, 622],
 [14, 622, 8],
 [14, 622, 8, 623],
 [14, 622, 8, 623, 415],
 [14, 622, 8, 623, 415, 416],
 [624, 2],
 [624, 2, 51],
 [624, 2, 51, 417],
 [624, 2, 51, 417, 418],
 [23, 625],
 [23, 625, 39],
 [23, 625, 39, 32],
 [23, 625, 39, 32, 3],
 [23, 625, 39, 32, 3, 106],
 [23, 625, 39, 32, 3, 106, 157],
 [23, 625, 39, 32, 3, 106, 157, 3],
 [23, 625, 39, 32, 3, 106, 157, 3, 243],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23, 39],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23, 39, 5],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23, 39, 5, 627],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23, 39, 5, 627, 419],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23, 39, 5, 627, 419, 27],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23, 39, 5, 627, 419, 27, 3],
 [23, 625, 39, 32, 3, 106, 157, 3, 243, 626, 23, 39, 5, 627, 419, 27, 3, 628],
 [23,
  625,
  39,
  32,
  3,
  106,
  157,
  3,
  243,
  

In [ ]:
setence_token_train = input_sequences_train[5] # [1, 1561, 5, 129, 34]
sentence_train = []
for token in setence_token_train:
    sentence_train.append(list((tokenizer.word_index).keys())[list((tokenizer.word_index).values()).index(token)])
print(sentence_train)

['honey', 'you']


In [ ]:
setence_token_test = input_sequences_test[6] # [1, 1561, 5, 129, 34]
sentence_test = []
for token in setence_token_test:
    sentence_test.append(list((tokenizer.word_index).keys())[list((tokenizer.word_index).values()).index(token)])
print(sentence_test)

['i', 'do', 'i', 'really', 'do']


In [ ]:
max_len_train = max([len(x) for x in input_sequences_train])

In [ ]:
max_len_train

137

In [ ]:
max_len_test = max([len(x) for x in input_sequences_test])

In [ ]:
max_len_test

61

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [ ]:
input_sequences_train

array([[   0,    0,    0, ...,    0,   14,  622],
       [   0,    0,    0, ...,   14,  622,    8],
       [   0,    0,    0, ...,  622,    8,  623],
       ...,
       [   0,    0,    0, ...,  350,  162,   88],
       [   0,    0,    0, ...,  162,   88,   12],
       [   0,    0,    0, ...,   88,   12, 1329]], dtype=int32)

In [ ]:
input_sequences_test = pad_sequences(input_sequences_test, maxlen = max_len_test, padding='pre')

Array ‘X’ is assigned the values of all rows in array ‘input_sequences’ except the last column. This means that ‘X’ contains all tokens in each array except the last one, which represents the input context. The ‘y’ array is assigned the values of the last column in the ‘input_sequences’ array that represents the target or the predicted word.

In [ ]:
input_sequences_test

array([[  0,   0,   0, ...,   0,   2,  21],
       [  0,   0,   0, ...,   2,  21, 212],
       [  0,   0,   0, ...,  21, 212,   9],
       ...,
       [  0,   0,   0, ..., 154,   1, 117],
       [  0,   0,   0, ...,   1, 117,  11],
       [  0,   0,   0, ..., 117,  11,  57]], dtype=int32)

In [ ]:
X = input_sequences[:,:-1]
y = input_sequences[:,-1]

In [ ]:
X.shape

(5278, 136)

In [ ]:
y.shape

(5278,)

In [ ]:
x = input_sequences_test[:,:-1]
Y = input_sequences_test[:,-1]

In [ ]:
x.shape

(1297, 60)

In [ ]:
Y.shape

(1297,)

In [ ]:
total_words_train

1318

IndexError: index 1330 is out of bounds for axis 1 with size 411

In [ ]:
total_words_train

1318

In [ ]:
y.shape

(5278, 1330)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
total_words_train

1318

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 136, 100)          131800    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 1318)              199018    
                                                                 
Total params: 481418 (1.84 MB)
Trainable params: 481418 (1.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X,y,epochs=100)

Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [ ]:
import time
text = "This article was about the Take"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=136, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 [==============================] - 0s 20ms/step
This article was about the Take back
1/1 [==============================] - 0s 19ms/step
This article was about the Take back the
1/1 [==============================] - 0s 19ms/step
This article was about the Take back the night
1/1 [==============================] - 0s 19ms/step
This article was about the Take back the night march
1/1 [==============================] - 0s 26ms/step
This article was about the Take back the night march this
1/1 [==============================] - 0s 19ms/step
This article was about the Take back the night march this was
1/1 [==============================] - 0s 18ms/step
This article was about the Take back the night march this was a
1/1 [==============================] - 0s 20ms/step
This article was about the Take back the night march this was a fourpart
1/1 [==============================] - 0s 18ms/step
This article was about the Take back the night march this was a fourpart series
1/1 [============

In [ ]:
import time
text = "Miranda"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=136, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 [==============================] - 0s 32ms/step
Miranda priestlys
1/1 [==============================] - 0s 21ms/step
Miranda priestlys office
1/1 [==============================] - 0s 26ms/step
Miranda priestlys office shes
1/1 [==============================] - 0s 20ms/step
Miranda priestlys office shes not
1/1 [==============================] - 0s 19ms/step
Miranda priestlys office shes not in
1/1 [==============================] - 0s 29ms/step
Miranda priestlys office shes not in ill
1/1 [==============================] - 0s 20ms/step
Miranda priestlys office shes not in ill leave
1/1 [==============================] - 0s 21ms/step
Miranda priestlys office shes not in ill leave word
1/1 [==============================] - 0s 21ms/step
Miranda priestlys office shes not in ill leave word in
1/1 [==============================] - 0s 21ms/step
Miranda priestlys office shes not in ill leave word in the
